# Блок 2. Работа с Hive

Cкачиваем данные, затем заливаем их в hdfs:

<pre>bash-5.0# (base) <font color="#26A269"><b>alex@alex-pc</b></font>:<font color="#12488B"><b>~/project/made/3_sem/ML_BD/HW2/HW2</b></font>$ 
docker cp ./artists.csv docker-hadoop-hive-parquet_namenode_1:/artists.csv
</pre>

<pre>bash-5.0# (base) <font color="#26A269"><b>alex@alex-pc</b></font>:<font color="#12488B"><b>~</b></font>$ docker exec -it docker-hadoop-hive-parquet_namenode_1  /bin/bash
root@26d99d579f51:/# hdfs dfs -ls
ls: `.&apos;: No such file or directory
root@26d99d579f51:/# hdfs dfs -ls /
Found 3 items
drwxr-xr-x   - root supergroup          0 2022-10-26 12:02 /rmstate
drwxrwxr-x   - root supergroup          0 2022-10-26 12:02 /tmp
drwxr-xr-x   - root supergroup          0 2022-10-26 12:02 /user
root@26d99d579f51:/# hdfs dfs -ls /user
Found 2 items
drwxr-xr-x   - root supergroup          0 2022-10-26 12:02 /user/hive
drwxr-xr-x   - hue  hue                 0 2022-10-26 12:02 /user/hue
root@26d99d579f51:/# hdfs dfs -ls /user/root
ls: `/user/root&apos;: No such file or directory
root@26d99d579f51:/# hdfs dfs -ls /user/hive
Found 1 items
drwxrwxr-x   - root supergroup          0 2022-10-26 12:02 /user/hive/warehouse
root@26d99d579f51:/# hdfs dfs -ls /
Found 3 items
drwxr-xr-x   - root supergroup          0 2022-10-26 12:02 /rmstate
drwxrwxr-x   - root supergroup          0 2022-10-26 12:02 /tmp
drwxr-xr-x   - root supergroup          0 2022-10-26 12:02 /user
root@26d99d579f51:/# hdfs dfs -mkdir /data
root@26d99d579f51:/# ls
artists.csv  bin  boot	dev  entrypoint.sh  etc  hadoop  hadoop-data  home  lib  lib64	media  mnt  opt  proc  root  run  run.sh  sbin	srv  sys  tmp  usr  var
root@26d99d579f51:/# hdfs dfs -mkdir /data/artists
root@26d99d579f51:/# hdfs dfs -copyFromLocal artists.csv /data/artists/
root@26d99d579f51:/# ^D
</pre>

<pre>(base) <font color="#26A269"><b>alex@alex-pc</b></font>:<font color="#12488B"><b>~/project/made/3_sem/ML_BD/HW2</b></font>$ docker exec -it docker-hadoop-hive-parquet_hive-server_1  /bin/bash
root@fd09d00c8449:/opt# hive
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.7.4/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in file:/opt/hive/conf/hive-log4j2.properties Async: true
Hive-on-MR is deprecated in Hive 2 and may not be available in the future versions. Consider using a different execution engine (i.e. spark, tez) or using Hive 1.X releases.
hive&gt; 
</pre>

In [7]:
import pandas as pd

df = pd.read_csv("../artists.csv")
df.head()

/tmp/ipykernel_1195880/3971229544.py:2: DtypeWarning: Columns (2,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../artists.csv")


,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,United States,United States,rock; alternative rock; 80s; 90s; rap; metal; ...,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,United States,Barbados; United States,pop; dance; hip hop; reggae; contemporary r b;...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,United States,United States,turkish; rap; american; hip-hop; hip hop; hiph...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0,False


# Задание 1
Сделать таблицу artists в Hive и вставить туда значения, используя датасет
https://www.kaggle.com/pieca111/music-artists-popularity - 15 баллов

```
DROP TABLE IF EXISTS artists;

CREATE EXTERNAL TABLE artists (
    mbid             string,
    artist_mb        string,
    artist_lastfm    string,
    country_mb       string,
    country_lastfm   string,
    tags_mb          string,
    tags_lastfm      string,
    listeners_lastfm int,
    scrobbles_lastfm int,
    ambiguous_artist boolean
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '/data/artists'
TBLPROPERTIES("skip.header.line.count"="1");
```

# Задание 2
Используя Hive, найти (команды и результаты записать в файл и добавить в
репозиторий):

## a) Исполнителя с максимальным числом скробблов - 5 баллов

```
SELECT artist_mb 
FROM artists
WHERE scrobbles_lastfm in (
    SELECT MAX(scrobbles_lastfm) FROM artists
);
```

<pre>Total MapReduce CPU Time Spent: 0 msec
OK
The Beatles
Time taken: 9.239 seconds, Fetched: 1 row(s)
</pre>

Ответ - The Beatles

## b) Самый популярный тэг на ластфм - 10 баллов

```
WITH tag_listeners_cnt_view AS (
    SELECT
        artist_lastfm,
        listeners_lastfm,
        tag_lastfm
    FROM artists
    LATERAL VIEW explode(split(tags_lastfm, '; ')) exploded AS tag_lastfm
)

SELECT
    tag_lastfm,
    sum(listeners_lastfm) AS cnt_listeners_lastfm
FROM tag_listeners_cnt_view
GROUP BY tag_lastfm
SORT BY cnt_listeners_lastfm DESC
LIMIT 1;
```

<pre>OK
seen live	4044163707
Time taken: 9.493 seconds, Fetched: 1 row(s)
</pre>

Ответ - seen live

__Примечачние:__  
Для вычисления популярности использовалось поле listeners_lastfm без скробблов.

## c) Самые популярные исполнители 10 самых популярных тегов ластфм - 10 баллов

```
WITH 
    tag_listeners_cnt_view AS (
        SELECT
            listeners_lastfm,
            tag_lastfm
        FROM artists
        LATERAL VIEW explode(split(tags_lastfm, '; ')) exploded AS tag_lastfm
    ),
    top_10_tag AS (
        SELECT
            tag_lastfm,
            sum(listeners_lastfm) AS cnt_listeners_lastfm
        FROM tag_listeners_cnt_view
        GROUP BY tag_lastfm
        SORT BY cnt_listeners_lastfm DESC
        LIMIT 10
    ),
    artists_tags AS (
        SELECT
            mbid,
            artist_lastfm,
            listeners_lastfm,
            tag_lastfm
        FROM artists
        LATERAL VIEW explode(split(tags_lastfm, '; ')) exploded AS tag_lastfm
    ),
    mbid_top AS (
        SELECT 
            /*+ MAPJOIN(top_10_tag) */
            mbid,
            artist_lastfm,
            listeners_lastfm
        FROM artists_tags AS a_t
        INNER JOIN top_10_tag AS t ON t.tag_lastfm = a_t.tag_lastfm
        GROUP BY mbid, artist_lastfm, listeners_lastfm
    )
SELECT
    artist_lastfm,
    sum(listeners_lastfm) AS cnt_listeners_lastfm
FROM mbid_top
GROUP BY artist_lastfm
SORT BY cnt_listeners_lastfm DESC
LIMIT 10;
```

<pre>OK
Phoenix	28424886
Chris Brown	13124285
John Williams	9627944
Passenger	6499619
Bliss	6380338
Feist	5900148
Plan B	5507433
Coldplay	5381567
Jet	5262465
Sebastian	4885461
Time taken: 24.651 seconds, Fetched: 10 row(s)
</pre>

## d) Любой другой инсайт на ваше усмотрение - 10 баллов

Мы ни разу не использовали поля стран.
Вывести топ 10 исполнителей, согласно их полурности listeners_lastfm в стране "United Kingdom":

```
SELECT
    artist_lastfm,
    sum(listeners_lastfm) as cnt_listeners_lastfm
FROM artists
GROUP BY artist_lastfm, country_mb
HAVING country_mb = "United Kingdom"
SORT BY cnt_listeners_lastfm DESC
LIMIT 10;
```

<pre>OK
Coldplay	5381567
Radiohead	4732528
Muse	4089612
Queen	4023379
The Rolling Stones	3798330
The Beatles	3674017
Oasis	3592076
Gorillaz	3577998
Arctic Monkeys	3501680
David Bowie	3354711
Time taken: 4.452 seconds, Fetched: 10 row(s)
</pre>